In [1]:
%pip install -q --no-cache-dir airavata-python-sdk[notebook]
import airavata_jupyter_magic


Note: you may need to restart the kernel to use updated packages.

Loaded airavata_jupyter_magic (2.0.12) 
(current runtime = local)

  %authenticate                      -- Authenticate to access high-performance runtimes.
  %request_runtime <rt> [args]       -- Request a runtime named <rt> with configuration <args>. Call multiple times to request multiple runtimes.
  %restart_runtime <rt>              -- Restart runtime <rt>. Run this if you install new dependencies or if the runtime hangs.
  %stop_runtime <rt>                 -- Stop runtime <rt> when no longer needed.
  %switch_runtime <rt>               -- Switch active runtime to <rt>. All subsequent executions will use this runtime.
  %%run_on <rt>                      -- Force a cell to always execute on <rt>, regardless of the active runtime.
  %stat_runtime <rt>                 -- Show the status of runtime <rt>.
  %copy_data <r1:file1> <r2:file2>   -- Copy <file1> in <r1> to <file2> in <r2>.



In [2]:
%authenticate
%request_runtime hpc_cpu --file=cybershuttle.yml --walltime=60 --use=NeuroData25VC1:cloud,expanse:shared,anvil:shared

# code below doesn't work, I believe it's in an error in the airavata code

# import os
# directory = "pictures"
# for filename in os.listdir(directory):
#     f = os.path.join(directory, filename)
#     if os.path.isfile(f):
#         image_path = f
#         %copy_data target=local:{image_path} source=hpc_cpu:{os.path.basename(your_path)}


%switch_runtime hpc_cpu

Output()

Authenticated.

Requesting runtime=hpc_cpu
cpuCount: 4
experimentName: CS_Agent
group: Default
libraries:
- python=3.10
- pip
memory: 0
mounts:
- cybershuttle-reference:/cybershuttle_data/cybershuttle-reference
nodeCount: 1
pip: []
queue: cloud
remoteCluster: NeuroData25VC1
wallTime: 60

Requested runtime=hpc_cpu. state=CONFIGURING_WORKSPACE
Switched to runtime=hpc_cpu.


In [3]:
%pip install transformers torch ipywidgets pillow

⠸ Connecting to=hpc_cpu... status=CONNECTEDWORKSPACEG_WORKSPACE
Note: you may need to restart the kernel to use updated packages.


In [4]:
import time
import torch
from transformers import AutoImageProcessor, ResNetForImageClassification
# from datasets import load_dataset
from PIL import Image
import tracemalloc
import subprocess
import gc
import os
import platform
from io import BytesIO
import requests

def get_process_memory():
    """Get memory usage in MB using OS-specific commands"""
    if platform.system() == "Linux":
        with open('/proc/self/status') as f:
            for line in f:
                if 'VmRSS' in line:
                    return int(line.split()[1]) / 1024  # Convert KB to MB
    else:
        #shouldn't be needed because I'm using WSL on local and Linux in ICE but still
        import resource
        return resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

def get_cpu_usage():
    """Get CPU utilization as a percentage without psutil."""
    if platform.system() == "Linux" or platform.system() == "Darwin":  # macOS/Linux
        with open('/proc/stat', 'r') as f:
            lines = f.readlines()
            cpu_line = lines[0].split()
            total_time = sum(map(int, cpu_line[1:]))  # Sum of all time components
            idle_time = int(cpu_line[4])  # Idle time
            return 100 * (1 - (idle_time / total_time))  # CPU usage percentage

    elif platform.system() == "Windows":  # Windows method using WMIC
        try:
            output = subprocess.check_output("wmic cpu get loadpercentage", shell=True)
            return float(output.split()[1])
        except Exception:
            return 0.0  # Fallback if command fails

    return 0.0  # Default if OS is unsupported

def track_performance(func):
    def wrapper(*args, **kwargs):
        gc.collect()  # Request garbage collection
        
        tracemalloc.start()  # Start memory tracking
        start_memory = get_process_memory()
        start_time = time.time()
        start_cpu = get_cpu_usage()  # Capture initial CPU usage

        result = func(*args, **kwargs)  # Execute function
        
        end_time = time.time()  # Capture end time
        end_memory = get_process_memory()
        end_cpu = get_cpu_usage()  # Capture final CPU usage
        current, peak = tracemalloc.get_traced_memory()
        
        tracemalloc.stop()  # Stop memory tracking

        print("\n=== Performance Metrics ===")
        print(f"Execution Time: {end_time - start_time:.2f} seconds")
        print(f"Memory Usage: {end_memory - start_memory:.2f} MB")
        print(f"Peak Memory During Execution: {peak / 1024 / 1024:.2f} MB")
        print(f"CPU Utilization: {end_cpu:.2f}%")
        if torch.cuda.is_available():
            print(f"GPU Memory Allocated: {torch.cuda.memory_allocated() / 1024 / 1024:.2f} MB")
            print(f"GPU Memory Reserved: {torch.cuda.memory_reserved() / 1024 / 1024:.2f} MB")
        print("========================\n")

        return result
    return wrapper

def classify_image(image_path, processor, model):
    # https://huggingface.co/docs/datasets/en/loading

    # dataset = load_dataset("huggingface/cats-image")
    # imagePath = dataset["test"]["image"][0]  # Get the first image from the test set
    # print(imagePath)
    # pil_image = Image.open(imagePath)
    if isinstance(image_path, Image.Image):
        pil_image = image_path
    else:
        pil_image = Image.open(image_path)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = model.to(device)

    inputs = processor(pil_image, return_tensors="pt") # image processing needed for model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    
    with torch.no_grad():
        logits = model(**inputs).logits # prediction for what image is done by model

    predicted_label = logits.argmax(-1).item()
    return model.config.id2label[predicted_label] # return the label of the image

@track_performance
def main():    
    url = "https://janakiev.com/assets/python-pilow-download-image_files/photo-1465056836041-7f43ac27dcb5.jpg" # change this url to an image you want (local images not supported because the copy command doesn't work)
    
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    
    processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")
    model = ResNetForImageClassification.from_pretrained("microsoft/resnet-50")
    
    prediction = classify_image(img, processor, model)
    print(f"Predicted class for downloaded image: {prediction}")

main()

⠹ Connecting to=hpc_cpu... status=CONNECTED


Predicted class for downloaded image: lakeside, lakeshore

=== Performance Metrics ===
Execution Time: 1.27 seconds
Memory Usage: 64.67 MB
Peak Memory During Execution: 4.25 MB
CPU Utilization: 0.34%
